# Lucena - Data Collection

## Finding Realease with most bugs

In [20]:
import json
from pprint import pprint
from os import listdir
from os.path import isfile, join
import operator

mypath = "issue_LUCENE"

onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) and f.endswith(".json") ]
version_count = {}

for f in onlyfiles:
    with open(join(mypath,f)) as data_file:   
        data = json.load(data_file)
        if (data["fields"]["issuetype"]["name"] == "Bug" and 
            data["fields"]["status"]["name"] == "Closed" and 
            data["fields"]["resolution"]["name"] == "Fixed"):
    
            versions = data["fields"]["versions"]
            for v in versions:
                if version_count.has_key(v["name"]):
                    version_count[v["name"]].append(data["key"])
                else:
                    version_count[v["name"]] = [data["key"]]
        


From the dictionary with the bugs associated to each release we extract the buggiest version and its bugs.

In [21]:
max = 0
buggy_version = ""

for k,v in version_count.iteritems():
    if len(v) > max:
        max = len(v)
        buggy_version = k
        
bugs = version_count[buggy_version]

print "Version with most closed bugs:", buggy_version , "with", len (version_count[buggy_version]), "bugs."

Version with most closed bugs: 4.0-ALPHA with 179 bugs.


For some reason shortlog was not working. #git shortlog -s -n

In [17]:
import sh
import re

#sh.git.clone("https://github.com/apache/lucene-solr.git")
git = sh.git.bake(_cwd='lucene-solr')

git.checkout("tags/lucene_solr_4_0_0_ALPHA")


files = filter(None,git("ls-files").split("\n"))

print "Reading repository..."

#git shortlog -s -n
#print sh.uniq(sh.sort(git.log("--format=format:%an", "README.txt")), "-c")

table = {}

for f in files:
    contributors_data = filter(None, sh.uniq(sh.sort(git.log("--format=format:%an", f)), "-c").split("\n"))
    contributors = []
    total = 0
    max_ownership = 0
    minor = 0
    major = 0
    
    for a in contributors_data:
        num = int(re.search("[0-9]+", a).group(0))
        name = re.search("([A-z]+\s*)+", a).group(0)
        total += num
        max_ownership = num if num > max_ownership else max_ownership
        contributors.append((name,num))
    
    for a in contributors:

        if a[1] * 1.0 / total >= 0.05:
            major += 1
        else:
            minor += 1
    
    table[f]= {"minor": minor, "major": major, "total": minor + major, "ownership": max_ownership / total * 100, "bug_count":0}
    #print table[f]

print table

print "finish"


Reading repository...
{u'lucene/queryparser/src/java/org/apache/lucene/queryparser/flexible/standard/config/FieldBoostMapFCListener.java': {'bug_count': 0, 'major': 2, 'total': 2, 'ownership': 0, 'minor': 0}, u'lucene/analysis/common/src/java/org/apache/lucene/analysis/ngram/EdgeNGramTokenizer.java': {'bug_count': 0, 'major': 2, 'total': 2, 'ownership': 0, 'minor': 0}, u'lucene/core/src/java/org/apache/lucene/document/package.html': {'bug_count': 0, 'major': 3, 'total': 3, 'ownership': 0, 'minor': 0}, u'lucene/core/src/java/org/apache/lucene/search/BooleanScorer2.java': {'bug_count': 0, 'major': 3, 'total': 3, 'ownership': 0, 'minor': 0}, u'solr/example/exampledocs/books.json': {'bug_count': 0, 'major': 2, 'total': 2, 'ownership': 0, 'minor': 0}, u'lucene/core/src/test/org/apache/lucene/util/TestDoubleBarrelLRUCache.java': {'bug_count': 0, 'major': 2, 'total': 2, 'ownership': 0, 'minor': 0}, u'lucene/core/src/test/org/apache/lucene/index/TestCheckIndex.java': {'bug_count': 0, 'major': 